# Lab 02: Training a Custom Model


**Objective of this lab**: training a small custom model on the Tiny-ImageNet dataset.

## Dataset preparation

In [1]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip tiny-imagenet-200.zip -d tiny-imagenet

Output streaming troncato alle ultime 5000 righe.
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_3979.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_3963.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_7199.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_2752.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_9687.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_9407.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_3603.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_3412.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_6982.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_8496.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_7332.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_9241.JPEG  
  inflating: tiny-imagenet/tiny-imagenet-200/val/images/val_4196.JPEG  
  inflating: t

We need to adjust the format of the val split of the dataset to be used with ImageFolder.

In [2]:
import os
import shutil

with open('tiny-imagenet/tiny-imagenet-200/val/val_annotations.txt') as f:
    for line in f:
        fn, cls, *_ = line.split('\t')
        os.makedirs(f'tiny-imagenet/tiny-imagenet-200/val/{cls}', exist_ok=True)

        shutil.copyfile(f'tiny-imagenet/tiny-imagenet-200/val/images/{fn}', f'tiny-imagenet/tiny-imagenet-200/val/{cls}/{fn}')

shutil.rmtree('tiny-imagenet/tiny-imagenet-200/val/images')

In [5]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

transform = T.Compose([
    T.Resize((224, 224)),  # Resize to fit the input dimensions of the network
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# root/{classX}/x001.jpg

tiny_imagenet_dataset_train = ImageFolder(root='tiny-imagenet/tiny-imagenet-200/train', transform=transform)
tiny_imagenet_dataset_val = ImageFolder(root='tiny-imagenet/tiny-imagenet-200/val', transform=transform)

In [6]:
print(f"Length of train dataset: {len(tiny_imagenet_dataset_train)}")
print(f"Length of val dataset: {len(tiny_imagenet_dataset_val)}")

# The following code also checks the number of samples per class
from collections import Counter

class_counts = Counter([target for _, target in tiny_imagenet_dataset_val])
for class_label, count in class_counts.items():
    print(f"Class {class_label}: {count} entries")


Length of train dataset: 100000
Length of val dataset: 10000
Class 0: 50 entries
Class 1: 50 entries
Class 2: 50 entries
Class 3: 50 entries
Class 4: 50 entries
Class 5: 50 entries
Class 6: 50 entries
Class 7: 50 entries
Class 8: 50 entries
Class 9: 50 entries
Class 10: 50 entries
Class 11: 50 entries
Class 12: 50 entries
Class 13: 50 entries
Class 14: 50 entries
Class 15: 50 entries
Class 16: 50 entries
Class 17: 50 entries
Class 18: 50 entries
Class 19: 50 entries
Class 20: 50 entries
Class 21: 50 entries
Class 22: 50 entries
Class 23: 50 entries
Class 24: 50 entries
Class 25: 50 entries
Class 26: 50 entries
Class 27: 50 entries
Class 28: 50 entries
Class 29: 50 entries
Class 30: 50 entries
Class 31: 50 entries
Class 32: 50 entries
Class 33: 50 entries
Class 34: 50 entries
Class 35: 50 entries
Class 36: 50 entries
Class 37: 50 entries
Class 38: 50 entries
Class 39: 50 entries
Class 40: 50 entries
Class 41: 50 entries
Class 42: 50 entries
Class 43: 50 entries
Class 44: 50 entries
Clas

In [103]:
import torch

train_loader = torch.utils.data.DataLoader(tiny_imagenet_dataset_train, batch_size=256+128, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(tiny_imagenet_dataset_val, batch_size=128, shuffle=False)

## Custom model definition

In [108]:
class CustomNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Primo blocco: 1 conv
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.maxpool1 = nn.MaxPool2d(8)
        self.avgpool1 = nn.AvgPool2d(8)
        self.alpha1 = nn.Parameter(torch.tensor(0.5))

        # Secondo blocco: 2 conv
        self.conv2a = nn.Conv2d(32, 64, 3, padding=1)
        self.conv2b = nn.Conv2d(64, 64, 3, padding=1)
        self.maxpool2 = nn.MaxPool2d(4)
        self.avgpool2 = nn.AvgPool2d(4)
        self.alpha2 = nn.Parameter(torch.tensor(0.5))

        # Terzo blocco: 3 conv
        self.conv3a = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3b = nn.Conv2d(128, 128, 3, padding=1)
        self.conv3c = nn.Conv2d(128, 128, 3, padding=1)
        self.maxpool3 = nn.MaxPool2d(2)
        self.avgpool3 = nn.AvgPool2d(2)
        self.alpha3 = nn.Parameter(torch.tensor(0.5))

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1152, 512)  # aggiornato
        self.fc2 = nn.Linear(512, 200)

    def forward(self, x):
        # Primo blocco
        x = self.conv1(x).relu()
        x1 = self.maxpool1(x)
        x2 = self.avgpool1(x)
        alpha1 = torch.clamp(self.alpha1, 0.0, 1.0)

        x = (1 - self.alpha1) * x1 + self.alpha1 * x2

        # Secondo blocco
        x = self.conv2a(x).relu()
        x = self.conv2b(x).relu()
        x1 = self.maxpool2(x)
        x2 = self.avgpool2(x)
        alpha2 = torch.clamp(self.alpha2, 0.0, 1.0)
        x = (1 - self.alpha2) * x1 + self.alpha2 * x2

        # Terzo blocco
        x = self.conv3a(x).relu()
        x = self.conv3b(x).relu()
        x = self.conv3c(x).relu()
        x1 = self.maxpool3(x)
        x2 = self.avgpool3(x)
        alpha3 = torch.clamp(self.alpha3, 0.0, 1.0)
        x = (1 - self.alpha3) * x1 + self.alpha3 * x2

        x = self.flat(x)
        x = self.fc1(x).relu()
        x = self.fc2(x)
        return x


In [109]:
def train(epoch, model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        # cleaning gradiets
        optimizer.zero_grad()
        # calculating previsions
        outputs = model(inputs)
        # calc the loss
        loss = criterion(outputs, targets)
        # calc gradient
        loss.backward()
        # update the model
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print(f"{batch_idx}/{len(train_loader)}-[{(float(batch_idx)/float(len(train_loader))*100.0):.2f}%]\tloss: {loss.item():.4f}")

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    print(f'Train Epoch: {epoch} Loss: {train_loss:.6f} Acc: {train_accuracy:.2f}%')

In [110]:
# Validation loop
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0

    correct, total = 0, 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.cuda(), targets.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    val_loss = val_loss / len(val_loader)
    val_accuracy = 100. * correct / total

    print(f'Validation Loss: {val_loss:.6f} Acc: {val_accuracy:.2f}%')
    return val_accuracy

## Putting everything together

In [111]:
model = CustomNet().cuda() # separated to avoid resetting the weights

In [112]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()


1113

In [114]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)


best_acc = 0

# Run the training process for {num_epochs} epochs
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    print(f"""Epoch {epoch} with learning rate: {optimizer.param_groups[0]['lr']}\n-----------------------------""")
    train(epoch, model, train_loader, criterion, optimizer)

    # At the end of each training iteration, perform a validation step
    val_accuracy = validate(model, val_loader, criterion)

    scheduler.step(val_accuracy)

    # Best validation accuracy
    best_acc = max(best_acc, val_accuracy)

    print(f"alpha1:{model.alpha1.item():.4f}\t alpha2:{model.alpha2.item():.4f}\t alpha3:{model.alpha3.item():.4f}")
print(f'Best validation accuracy: {best_acc:.2f}%')


Epoch 1 with learning rate: 0.1
-----------------------------
0/261-[0.00%]	loss: 4.1898
1/261-[0.38%]	loss: 4.0892
2/261-[0.77%]	loss: 4.0666
3/261-[1.15%]	loss: 4.0747
4/261-[1.53%]	loss: 4.1364
5/261-[1.92%]	loss: 4.1094
6/261-[2.30%]	loss: 4.0777
7/261-[2.68%]	loss: 4.0158
8/261-[3.07%]	loss: 4.0680
9/261-[3.45%]	loss: 4.0755
10/261-[3.83%]	loss: 4.0812
11/261-[4.21%]	loss: 4.2201
12/261-[4.60%]	loss: 4.2157
13/261-[4.98%]	loss: 4.0572
14/261-[5.36%]	loss: 4.2622
15/261-[5.75%]	loss: 3.9489
16/261-[6.13%]	loss: 4.1110
17/261-[6.51%]	loss: 4.1868
18/261-[6.90%]	loss: 4.0638
19/261-[7.28%]	loss: 4.2287
20/261-[7.66%]	loss: 4.1381
21/261-[8.05%]	loss: 4.1741
22/261-[8.43%]	loss: 4.1035
23/261-[8.81%]	loss: 4.0902
24/261-[9.20%]	loss: 4.1035
25/261-[9.58%]	loss: 4.0461
26/261-[9.96%]	loss: 4.1470
27/261-[10.34%]	loss: 4.0485
28/261-[10.73%]	loss: 4.1064
29/261-[11.11%]	loss: 4.2932
30/261-[11.49%]	loss: 4.1696
31/261-[11.88%]	loss: 4.1160
32/261-[12.26%]	loss: 4.2387
33/261-[12.64%]	lo